<a href="https://colab.research.google.com/github/Annie29/AdventOfCode2018/blob/master/2018Advent4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Advent of Code, 2018
##Problem 4

From [adventofcode.com](https://adventofcode.com/2018/day/4)

Covering the walls, someone has spent an hour starting every midnight for the past few months secretly observing this guard post! They've been writing down the ID of the one guard on duty that night - the Elves seem to have decided that one guard was enough for the overnight shift - as well as when they fall asleep or wake up while at their post (your puzzle input).

For example, consider the following records, which have already been organized into chronological order:

* [1518-11-01 00:00] Guard #10 begins shift
* [1518-11-01 00:05] falls asleep
* [1518-11-01 00:25] wakes up
* [1518-11-01 00:30] falls asleep
* [1518-11-01 00:55] wakes up
* [1518-11-01 23:58] Guard #99 begins shift
* [1518-11-02 00:40] falls asleep
* [1518-11-02 00:50] wakes up
* [1518-11-03 00:05] Guard #10 begins shift
* [1518-11-03 00:24] falls asleep
* [1518-11-03 00:29] wakes up
* [1518-11-04 00:02] Guard #99 begins shift
* [1518-11-04 00:36] falls asleep
* [1518-11-04 00:46] wakes up
* [1518-11-05 00:03] Guard #99 begins shift
* [1518-11-05 00:45] falls asleep
* [1518-11-05 00:55] wakes up

Timestamps are written using year-month-day hour:minute format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), only the minute portion (00 - 59) is relevant for those events.

Visually, these records show that the guards are asleep at these times:

    Date   ID   Minute
                  000000000011111111112222222222333333333344444444445555555555
                  012345678901234567890123456789012345678901234567890123456789
    11-01  #10  .....####################.....#########################.....
    11-02  #99  ........................................##########..........
    11-03  #10  ........................#####...............................
    11-04  #99  ....................................##########..............
    11-05  #99  .............................................##########.....

The columns are Date, which shows the month-day portion of the relevant day; ID, which shows the guard on duty that day; and Minute, which shows the minutes during which the guard was asleep within the midnight hour. (The Minute column's header shows the minute's ten's digit in the first row and the one's digit in the second row.) Awake is shown as ., and asleep is shown as #.

Note that guards count as asleep on the minute they fall asleep, and they count as awake on the minute they wake up. For example, because Guard #10 wakes up at 00:25 on 1518-11-01, minute 25 is marked as awake.

If you can figure out the guard most likely to be asleep at a specific time, you might be able to trick that guard into working tonight so you can have the best chance of sneaking in. You have two strategies for choosing the best guard/minute combination.

Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

In the example above, Guard #10 spent the most minutes asleep, a total of 50 minutes (20+25+5), while Guard #99 only slept for a total of 30 minutes (10+10+10). Guard #10 was asleep most during minute 24 (on two days, whereas any other minute the guard was asleep was only seen on one day).

While this example listed the entries in chronological order, your entries are in the order you found them. You'll need to organize them before they can be analyzed.

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 10 * 24 = 240.)

Okay, the rote stuff:

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving four.txt to four.txt
User uploaded file "four.txt" with length 38305 bytes


In [0]:
with open('four.txt') as f:
  data = f.readlines()

In [0]:
#  Since the data is not guaraneteed to be in order:
data.sort()

Soooo, what to do?  First find the guard with the most sleep minutes.  (It's sorta silly, we should be looking at the percentage, but I s'pose this is complicated enough.)

That involves a lot of the parsing we did before.  The items in the split:

*   Date (probably not used)
*   Time
*  "Guard" or "falls" or "wakes"
*  guard number, preceded by #

Let's count the time with a dictionary.




In [13]:
current_guard = None
sleep_time = {}
sleep_start = None

for line in data:
  words = line.split()
  full_time = words[1].split(':')
  minutes = full_time[1][:2]
  time = int(minutes)
  
  action = words[2]
  
  if action == "Guard":  
    #  change the current guard
    current_guard = int(words[3][1:])
  elif action == "falls":
    sleep_start = time
  elif action == "wakes":  
    time_asleep = time - sleep_start
    if current_guard in sleep_time:
      old_time = sleep_time[current_guard]
      sleep_time[current_guard] += time_asleep
    else:
      sleep_time[current_guard] = time_asleep
  else:
    print ('Bad action ' + action)

sleep_time

{61: 148,
 113: 151,
 419: 439,
 661: 498,
 733: 570,
 739: 450,
 859: 408,
 997: 466,
 1163: 323,
 1367: 348,
 1663: 512,
 2113: 198,
 2221: 253,
 2297: 391,
 2383: 216,
 2609: 222,
 2713: 194,
 3203: 337,
 3391: 419,
 3407: 385}

That seems to figure out all the guards.

Let's find the one with the most time asleep.

In [16]:
max_time = -1
worst_guard = None

for guard, time in sleep_time.items():
  if time > max_time: 
    worst_guard = guard
    max_time = time
    
print (str(worst_guard) + ": " + str(max_time))
  
  

733: 570


So, guard 733 is our problem child.  

Now let's count times to find when they're asleep the most.

Could I have done this during the first loop?  Yeah.

Am I going to change a loop that works?  Nope.

Do I want to be sure I understand range?  Sure.

In [19]:
for i in range (3,6):
  print(str(i))


3
4
5


In [25]:
#  Find the times this guard has been asleep

times_asleep = [0 for x in range (60)]

for line in data:
  words = line.split()
  full_time = words[1].split(':')
  minutes = full_time[1][:2]
  time = int(minutes)
  
  action = words[2]
  
  if action == "Guard":  
    #  change the current guard
    current_guard = int(words[3][1:])
  elif action == "falls":
    sleep_start = time
  elif action == "wakes":  
    
    if current_guard == worst_guard:
      for i in range(sleep_start, time):
        times_asleep[i] += 1
      
# times_asleep

#  And then find the longest of all these times.

longest_time = -1
sleepiest_time = None

for i in range(len(times_asleep)):
  t = times_asleep[i]
  if t > longest_time:
    longest_time = t
    sleepiest_time = i
    
print(str(sleepiest_time)  + ": " + str(longest_time))

print("result = " + str(sleepiest_time * worst_guard))
  

48: 17
result = 35184


###Part B
Of all guards, which guard is most frequently asleep on the same minute?

In the example above, Guard #99 spent minute 45 asleep more than any other guard or minute - three times in total. (In all other cases, any guard spent any minute asleep at most twice.)

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 99 * 45 = 4455.)

Well, crud.  I probably should have tried to do part A in one loop.

Eh, let's take the second loop and adjust it to answer this part.  It shouldn't be too rough.


In [36]:
#  Find the times all guards have been asleep
guard_sleep_times = {}

for line in data:
  words = line.split()
  full_time = words[1].split(':')
  minutes = full_time[1][:2]
  time = int(minutes)
  
  action = words[2]
  
  if action == "Guard":  
    #  change the current guard
    current_guard = int(words[3][1:])
  elif action == "falls":
    sleep_start = time
  elif action == "wakes":  
    if current_guard not in guard_sleep_times:
      times_asleep = [0 for x in range(60)]
    else:
      times_asleep = guard_sleep_times[current_guard]
      
    for i in range(sleep_start, time):
      times_asleep[i] += 1
    guard_sleep_times[current_guard] = times_asleep
      
# times_asleep
#for k, v in guard_sleep_times.items():
#  print (str(k) + ": " + str(v))

#  And then find the longest of all these times.

longest_time = -1
sleepiest_time = None
sleepiest_guard = None

for guard, times_asleep in guard_sleep_times.items():
  for i in range(len(times_asleep)):
    t = times_asleep[i]
    if t > longest_time:
      longest_time = t
      sleepiest_time = i
      sleepiest_guard = guard
    
print(str(sleepiest_time)  + ": " + str(longest_time) + ", guard = " + str(sleepiest_guard))

print("result = " + str(sleepiest_time * sleepiest_guard))


38: 19, guard = 997
result = 37886
